# Part g): Analysis of real data 

### Import libraries

In [ ]:
import numpy as np
from imageio import imread
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import git
import sys
sys.path.append("../")
import functions as f
path_to_root = git.Repo(".", search_parent_directories=True).working_dir
sys.path.append(path_to_root)
plt.style.use('seaborn-v0_8-whitegrid')

### Import terrain data

In [ ]:
# Load the terrain
terrain1 = imread(path_to_root+"/data/SRTM_data_Norway_1.tif")
# Show the terrain
plt.figure()
plt.title("Terrain over Norway 1")
plt.imshow(terrain1, cmap="gray")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()
print(terrain1.shape)
print(type(terrain1))
terrain1[2000:3601,:].shape

### Analysis of terrain data

In [ ]:
# Load the terrain data
terrain_data = imread(path_to_root+"/data/SRTM_data_Norway_1.tif")

# Define the x and y coordinates
#x = np.arange(0, terrain_data[2000:3601,:].shape[1], 10)
x = np.arange(0, terrain_data.shape[1], 10)
#y = np.arange(0, terrain_data[2000:3601,:].shape[0], 10)
y = np.arange(0, terrain_data.shape[0], 10)
# Create a meshgrid of the x and y coordinates
xv, yv = np.meshgrid(x, y)

# Sample z
#zv = terrain_data[2000:3601:10, ::10]
zv = terrain_data[::10, ::10]


In [ ]:
fig = plt.figure()
#ax = fig.gca(projection='3d')
ax = fig.add_subplot(111, projection='3d')

# Plot the surface.
surf = ax.plot_surface(xv, yv, zv, cmap=cm.coolwarm,
linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(0,2000)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter("%.02f"))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
x = xv.flatten()
y = yv.flatten()
z = zv.flatten().reshape(-1, 1)
print(x.shape)
print(y.shape)
print(z.shape)

In [ ]:
# Create the heatmap
aspect_ratio = xv.shape[1] / yv.shape[0]
plt.figure(figsize=(5, 5 / aspect_ratio))

heatmap = plt.pcolormesh(xv, yv, zv, shading='auto', cmap='viridis')

# Add color bar
plt.colorbar(heatmap)

# Set labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Heatmap of sampled terrain data')
plt.gca().invert_yaxis()
# Show the plot
#f.save_to_results(filename = "sampled_data_heatmap.png")

# OLS Reproduction

In [ ]:
degrees = np.arange(1, 15)

beta_values = []
mse_values = np.zeros(len(degrees))
R2_values = np.zeros(len(degrees))

for i, degree in enumerate(degrees): 
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree)
    X = poly_features.fit_transform(np.column_stack((x, y)))

    # Split the data into training and test data
    X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=42)

    # Scale and center the data
    X_train, X_test = f.scale_train_test(train = X_train, test = X_test)
    z_train, z_test = f.scale_train_test(train = z_train, test = z_test)

    #Calculating OLSbeta, ztilde, mse and R2
    OLSbeta = f.beta_OLS(X_train, z_train)
    ztilde = f.z_predict(X_test, OLSbeta)
    mse = f.mse(z_test, ztilde)
    R2 = f.r2(z_test, ztilde)

    #Adding the values to the arrays

    beta_values.append(OLSbeta)
    mse_values[i] = mse
    R2_values[i] = R2

In [ ]:
# Plotting MSE and R2 scores
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(degrees, mse_values, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('MSE as a function of Polynomial Degree for OLS')

plt.subplot(1, 2, 2)
plt.plot(degrees, R2_values, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('R2')
plt.title('R2 as a function of Polynomial Degree for OLS')
plt.tight_layout()
plt.show()

## Retrieve the estimated z_values

In [ ]:
#use the latest prediction as it has the best score 
ztilde = f.z_predict(X, OLSbeta)

In [ ]:
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X = X_scaler.transform(X)

z_scaler = StandardScaler()
#z_train = z_scaler.fit_transform(z_train)
#z_test = z_scaler.transform(z_test)

In [ ]:
ztildev = ztilde.flatten()
print(ztilde.shape)
ztildev = ztilde.reshape(361, 181)
print(ztildev.shape)

In [ ]:
# Do we have to scale the betavalues back? 

In [ ]:
# Create the heatmap
#aspect_ratio = xv.shape[1] / yv.shape[0]
#plt.figure(figsize=(5, 5 / aspect_ratio))

heatmap = plt.pcolormesh(xv, yv, ztildev, cmap='viridis')

# Add color bar
plt.colorbar(heatmap)

# Set labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Heatmap of sampled terrain data')
plt.gca().invert_yaxis()

In [ ]:
for degree, values in enumerate(beta_values):
    print("degree", degree, "betavalues", len(values))
    print(values)
    print(" ")
    degrees = np.repeat(degree, len(values))
    plt.scatter(degrees, values)

plt.xlabel('Polynomial Degree')
plt.ylabel('Beta Values')
plt.title('Beta values as a function of a Polynomial Degree for OLS')


# Ridge reproduction

In [ ]:

degrees = np.arange(1, 15)

beta_ridge_values = []
mse_ridge_values = np.zeros(len(degrees))
R2_ridge_values = np.zeros(len(degrees))

lambda_values = np.logspace(-5, 3, 9)

for i, degree in enumerate(degrees): 
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree)
    X = poly_features.fit_transform(np.column_stack((x, y)))

    # Split the data into training and test data
    X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=42)

    # Scale and center the data
    X_train, X_test = f.scale_train_test(train = X_train, test = X_test)
    z_train, z_test = f.scale_train_test(train = z_train, test = z_test)

    mse_temp = np.zeros(len(lambda_values))
    for j, lambda_ in enumerate(lambda_values):
        #Calculating OLSbeta, ztilde, mse and R2
        Ridgebeta = f.beta_ridge(X_train, z_train, lambda_)
        ztilde = f.z_predict(X_test, Ridgebeta)
        mse = f.mse(z_test, ztilde)
        mse_temp[j] = mse

    j = np.argmin(mse_temp)
    Ridgebeta = f.beta_ridge(X_train, z_train, lambda_values[j])
    ztilde = f.z_predict(X_test, Ridgebeta)

    #Adding the values to the arrays
    beta_ridge_values.append(Ridgebeta)
    mse_ridge_values[i] = f.mse(z_test, ztilde)
    R2_ridge_values[i] = f.r2(z_test, ztilde)



In [ ]:
# Plotting MSE and R2 scores
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(degrees, mse_ridge_values, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('MSE as a function of Polynomial Degree for Ridge')

plt.subplot(1, 2, 2)
plt.plot(degrees, R2_ridge_values, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('R2')
plt.title('R2 as a function of Polynomial Degree for Ridge')
plt.tight_layout()
plt.show()

In [ ]:
for degree, values in enumerate(beta_ridge_values):
    print("degree", degree, "betavalues", len(values))
    print(values)
    print(" ")
    degrees = np.repeat(degree, len(values))
    plt.scatter(degrees, values)

plt.xlabel('Polynomial Degree')
plt.ylabel('Beta Values')
plt.title('Beta values as a function of a Polynomial Degree for Ridge')

## Retrieve the estimated z_values

In [ ]:
#use the latest prediction as it has the best score 
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X = X_scaler.transform(X)

z_scaler = StandardScaler()
z_scaler.fit(z_train)
ztilde = z_scaler.inverse_transform(ztilde)
ztilde = f.z_predict(X, Ridgebeta)

In [ ]:
ztildev = ztilde.flatten()
print(ztilde.shape)
ztildev = ztilde.reshape(361, 181)
print(ztildev.shape)

In [ ]:
# Create the heatmap
#aspect_ratio = xv.shape[1] / yv.shape[0]
#plt.figure(figsize=(5, 5 / aspect_ratio))

heatmap = plt.pcolormesh(xv, yv, ztildev, cmap='viridis')

# Add color bar
plt.colorbar(heatmap)

# Set labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Heatmap of sampled terrain data')
plt.gca().invert_yaxis()

# Lasso Reproduction

In [ ]:
degrees = np.arange(1, 9)

beta_lasso_values = []
mse_lasso_values = np.zeros(len(degrees))
R2_lasso_values = np.zeros(len(degrees))

lambda_values = np.logspace(-5, -3, 2)

for i, degree in enumerate(degrees): 
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree)
    X = poly_features.fit_transform(np.column_stack((x, y)))

    # Split the data into training and test data
    X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=42)

    # Scale and center the data
    X_train, X_test = f.scale_train_test(train = X_train, test = X_test)
    z_train, z_test = f.scale_train_test(train = z_train, test = z_test)
    
    mse_temp = np.zeros(len(lambda_values))

    for j, lambda_ in enumerate(lambda_values):
        # Create and fit the linear regression model
        model = Lasso(alpha = lambda_values[j], tol = 1e-2, selection = 'random', precompute=True, fit_intercept=False, max_iter=10000) #selection = "random"
        model.fit(X_train, z_train)

        # Make predictions for training and test data
        z_train_pred = model.predict(X_train)
        z_test_pred = model.predict(X_test)
        #print(z_train_pred.shape)

        # Compute mean squared error for training and test data
        #mse_train = f.mse(z_train, z_train_pred)
        mse_test = f.r2(z_test, z_test_pred)

        mse_temp[j] = mse_test

    j = np.argmin(mse_temp)
    print(lambda_values[j])
     # Create and fit the linear regression model with the optimal lambda value
    model = Lasso(alpha = lambda_values[j], tol = 1e-2, selection = 'random', precompute=True, fit_intercept=False, max_iter=10000)
    model.fit(X_train, z_train)

    # Make predictions for training and test data
    z_train_pred = model.predict(X_train)
    z_test_pred = model.predict(X_test)

    Lassobeta = model.coef_

    #Adding the values to the arrays
    beta_lasso_values.append(Lassobeta)
    mse_lasso_values[i] = f.mse(z_test, z_test_pred)
    R2_lasso_values[i] = f.r2(z_test, z_test_pred)


In [ ]:
# Plotting MSE and R2 scores
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(degrees, mse_lasso_values, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('MSE as a function of Polynomial Degree for Lasso')

plt.subplot(1, 2, 2)
plt.plot(degrees, R2_lasso_values, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('R2')
plt.title('R2 as a function of Polynomial Degree for Lasso')
plt.tight_layout()
plt.show()

In [ ]:
for degree, values in enumerate(beta_lasso_values):
    print("degree", degree, "betavalues", len(values))
    print(values)
    print(" ")
    degrees = np.repeat(degree, len(values))
    plt.scatter(degrees, values)

plt.xlabel('Polynomial Degree')
plt.ylabel('Beta Values')
plt.title('Beta values as a function of a Polynomial Degree for Lasso')

# Plotting OLS and Ridge together

In [ ]:
degrees = np.arange(1, 30)
# Plotting MSE and R2 scores
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(degrees, mse_values, marker='o', label='OLS')
plt.plot(degrees, mse_ridge_values, marker='o', label='Ridge')
#plt.plot(degrees, mse_lasso_values, marker='o', label='Lasso')
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('MSE as a function of Polynomial Degree')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(degrees, R2_values, marker='o', label='OLS')
plt.plot(degrees, R2_ridge_values, marker='o', label='Ridge')
#plt.plot(degrees, R2_lasso_values, marker='o', label='Lasso')
plt.xlabel('Polynomial Degree')
plt.ylabel('R2')
plt.title('R2 as a function of Polynomial Degree')
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
import numpy as np
from imageio import imread
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
plt.style.use('seaborn-v0_8-whitegrid')
import git
import sys
sys.path.append("../")
path_to_root = git.Repo(".", search_parent_directories=True).working_dir
sys.path.append(path_to_root+'/src'+'/')
import functions as f
import load_data as ld
plt.style.use('seaborn-v0_8-whitegrid')

# Bootstraping and Bias Variance tradeoff

In [ ]:
# Load the terrain data
terrain_data = imread(path_to_root+"/data/SRTM_data_Norway_1.tif")

# Define the x and y coordinates
x = np.arange(0, terrain_data.shape[1], 10)
y = np.arange(0, terrain_data.shape[0], 10)

xv, yv = np.meshgrid(x, y)

x= xv.flatten()
y= yv.flatten()
print(x.shape, y.shape)

# Sample z
#zv = terrain_data[2000:3601:10, ::10]
zv = terrain_data[::10, ::10]
z = zv.flatten().reshape(-1, 1)
print(z.shape)

In [ ]:
n_boostraps = 100
degree = np.arange(0, 15)

# Make data set
#x, y, z = ld.load_uniform_data(N_samples = 1000)

errortest = np.zeros(len(degree))
errortrain = np.zeros(len(degree))  
bias = np.zeros(len(degree))
variance = np.zeros(len(degree))

for i in degree:
    # Combine x transformation and model into one operation.
    #X = f.create_design_matrix(x, y, i)
    design_matrix = PolynomialFeatures(degree=i)
    X = design_matrix.fit_transform(np.column_stack((x, y)))

    # Splitting the data into training and test data
    X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2)

    # Scale and center the data
    X_train, X_test = f.scale_train_test(train = X_train, test = X_test)
    z_train, z_test = f.scale_train_test(train = z_train, test = z_test)

    # Fit the model
    ols = LinearRegression(fit_intercept=False)
    ridge = Ridge(alpha=0.0001, fit_intercept=False)   
    lasso = Lasso(alpha=0.0001, fit_intercept=False)

    model = ols # Change this variable if you want to use another model

    # The following (m x n_bootstraps) matrix holds the column vectors y_pred
    # for each bootstrap iteration.
    z_pred = np.zeros((z_test.shape[0], n_boostraps))

    for j in range(n_boostraps):
        X_, z_ = resample(X_train, z_train)

        # Evaluate the new model on the same test data each time.
        z_pred[:, j] = model.fit(X_, z_).predict(X_test).ravel() 

        # Note: Expectations and variances taken w.r.t. different training
        # data sets, hence the axis=1. Subsequent means are taken across the test data
        # set in order to obtain a total value, but before this we have error/bias/variance
        # calculated per data point in the test set.
        # Note 2: The use of keepdims=True is important in the calculation of bias as this 
        # maintains the column vector form. Dropping this yields very unexpected results.

    errortest[i] = np.mean( np.mean((z_test - z_pred)**2, axis=1, keepdims=True) )
    errortrain[i] = np.mean( np.mean((z_train - ols.fit(X_train, z_train).predict(X_train))**2, axis=1, keepdims=True ) ) # modelols.fit(X_train, z_train).predict(X_train))**2 ) ?
    bias[i] = np.mean( (z_test - np.mean(z_pred, axis=1, keepdims=True))**2 )
    variance[i] = np.mean( np.var(z_pred, axis=1, keepdims=True) )


plt.title(f'Bias variance tradeoff for OLS')
plt.xlabel('Model complexity (degree)')
plt.ylabel('Error')
plt.plot(degree, errortest, label='Error')
plt.plot(degree, bias, label='Bias')
plt.plot(degree, variance, label='Variance')
plt.legend()
plt.show()


In [ ]:
# Plot the data
plt.figure(figsize=(8, 6))
plt.plot(degree, errortrain, label="Training Sample", color='teal', linewidth=2)
plt.plot(degree, errortest, label="Test Sample", color='red', linewidth=2)

# Add labels and title
plt.xlabel("Model Complexity")
plt.ylabel("Prediction Error")
plt.title("Test and Training Error as a function of Model Complexity")

# Add legend
plt.legend()

# Show the plot
plt.show()


# Cross Validation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
# Load the terrain data
terrain_data = imread(path_to_root+"/data/SRTM_data_Norway_1.tif")

# Define the x and y coordinates
x = np.arange(0, terrain_data.shape[1], 10)
y = np.arange(0, terrain_data.shape[0], 10)

xv, yv = np.meshgrid(x, y)

x= xv.flatten()
y= yv.flatten()
print(x.shape, y.shape)

# Sample z
#zv = terrain_data[2000:3601:10, ::10]
zv = terrain_data[::10, ::10]
z = zv.flatten().reshape(-1, 1)
print(z.shape)

In [ ]:

# Load your data (ensure ld.load_uniform_data() is defined)
#x, y, z = ld.load_uniform_data()

# Define the range of polynomial degrees to test
degrees = np.arange(1, 10)  # Start from degree 1 to avoid intercept-only model

# Define the number of folds
k_folds = 10

# Initialize arrays to store the MSE values
mse_ols = np.zeros_like(degrees, dtype=float)
mse_ridge = np.zeros_like(degrees, dtype=float)
mse_lasso = np.zeros_like(degrees, dtype=float)

# Define lambda values for Ridge and Lasso
lambda_values = np.logspace(-5, 2, 8)  # From 1e-5 to 1e2

# Perform k-fold cross-validation for each degree
for i, degree in enumerate(degrees):
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree)
    X = poly_features.fit_transform(np.column_stack((x, y)))

    # No need to scale here; include scaling in the pipeline

    # OLS Model with Pipeline
    pipeline_ols = Pipeline([
        ('scaler', StandardScaler()),
        ('model', LinearRegression(fit_intercept=False))
    ])
    mse_ols[i] = -np.mean(cross_val_score(pipeline_ols, X, z, cv=k_folds, scoring='neg_mean_squared_error'))

    # Ridge Regression
    mse_temp_ridge = []
    for lambda_val in lambda_values:
        pipeline_ridge = Pipeline([
            ('scaler', StandardScaler()),
            ('model', Ridge(alpha=lambda_val, fit_intercept=False))
        ])
        mse = -np.mean(cross_val_score(pipeline_ridge, X, z, cv=k_folds, scoring='neg_mean_squared_error'))
        mse_temp_ridge.append(mse)
    mse_ridge[i] = np.min(mse_temp_ridge)
    j = np.argmin(mse_temp_ridge)
    print(f"Degree {degree}: Best lambda value for Ridge = {lambda_values[j]}")

    # Lasso Regression
    mse_temp_lasso = []
    for lambda_val in lambda_values:
        pipeline_lasso = Pipeline([
            ('scaler', StandardScaler()),
            ('model', Lasso(alpha=lambda_val, fit_intercept=False, max_iter=10000))
        ])
        mse = -np.mean(cross_val_score(pipeline_lasso, X, z, cv=k_folds, scoring='neg_mean_squared_error'))
        mse_temp_lasso.append(mse)
    mse_lasso[i] = np.min(mse_temp_lasso)
    j = np.argmin(mse_temp_lasso)
    print(f"Degree {degree}: Best lambda value for Lasso = {lambda_values[j]}")

# Plot the MSE values
plt.figure(figsize=(8, 6))
plt.plot(degrees, mse_ols, label='OLS')
plt.plot(degrees, mse_ridge, label='Ridge')
plt.plot(degrees, mse_lasso, label='Lasso')
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('MSE for OLS, Ridge, and Lasso')
plt.legend()
plt.show()
